## Importação de Bibliotecas

In [0]:
import unicodedata
from pyspark.sql import functions
from pyspark.sql.window import Window
from pyspark.sql.functions import udf
from pyspark.sql.functions import col, count, months_between, current_date, date_format, to_timestamp, lpad, regexp_replace

## Criação do DataFrame

In [0]:
df_produto = spark.createDataFrame(
    [
        ("001", "1", "Televisor"),
        ("002", "1", "Televisor"),
        ("003", "1", "Televisor"),
        ("004", "3", "Violao"),
        ("005", "5", "Xbox One"),
        ("006", "10", "Geladeira")
    ],
    ["cod_cli", "cod_prod", "nome"]
)

In [0]:
df_cliente = spark.createDataFrame(
    [
        ("001", "José", "Anápolis", "São Paulo", "01-09-1900"),
        ("02", "Igor", "Anápolis", "São Paulo", "11-09-1977"),
        ("3", "Leonardo", "Anápolis", "São Paulo", "21-12-2000"),
        ("04", "Humberto", "Pato Branco", "Rio Grande do Sul", "13-11-1964"),
        ("005", "Isaias", "Pato Branco", "Rio Grande do Sul", "07-07-2002"),
        ("6", "Lucas", "Taua", "Ceará", "05-09-1984")
    ],
    ["cod_cliente", "Nome", "Município", "Estado", "data_nascimento"]
)

In [0]:
df_cliente.show()

+-----------+--------+-----------+-----------------+---------------+
|cod_cliente|    Nome|  Município|           Estado|data_nascimento|
+-----------+--------+-----------+-----------------+---------------+
|        001|    José|   Anápolis|        São Paulo|     01-09-1900|
|         02|    Igor|   Anápolis|        São Paulo|     11-09-1977|
|          3|Leonardo|   Anápolis|        São Paulo|     21-12-2000|
|         04|Humberto|Pato Branco|Rio Grande do Sul|     13-11-1964|
|        005|  Isaias|Pato Branco|Rio Grande do Sul|     07-07-2002|
|          6|   Lucas|       Taua|            Ceará|     05-09-1984|
+-----------+--------+-----------+-----------------+---------------+



## Inicio do tratamento dos dados

In [0]:
df_cliente = df_cliente.withColumnRenamed('cod_cliente', 'cod_cli')

In [0]:
WindowSpec = Window.partitionBy("Município")
df_cliente = df_cliente.withColumn("qtd_municipio" ,
                                count("Município").over(WindowSpec))
df_cliente = df_cliente.sort("Estado")

In [0]:
df_cliente.show()

+-------+--------+-----------+-----------------+---------------+-------------+
|cod_cli|    Nome|  Município|           Estado|data_nascimento|qtd_municipio|
+-------+--------+-----------+-----------------+---------------+-------------+
|      6|   Lucas|       Taua|            Ceará|     05-09-1984|            1|
|     04|Humberto|Pato Branco|Rio Grande do Sul|     13-11-1964|            2|
|    005|  Isaias|Pato Branco|Rio Grande do Sul|     07-07-2002|            2|
|    001|    José|   Anápolis|        São Paulo|     01-09-1900|            3|
|     02|    Igor|   Anápolis|        São Paulo|     11-09-1977|            3|
|      3|Leonardo|   Anápolis|        São Paulo|     21-12-2000|            3|
+-------+--------+-----------+-----------------+---------------+-------------+



In [0]:
df_cliente = df_cliente.withColumn('Idade', (months_between(current_date(), to_timestamp(col("data_nascimento"),'dd-MM-yyyy')) / 12).cast('int'))

In [0]:
df_cliente.show()

+-------+--------+-----------+-----------------+---------------+-------------+-----+
|cod_cli|    Nome|  Município|           Estado|data_nascimento|qtd_municipio|Idade|
+-------+--------+-----------+-----------------+---------------+-------------+-----+
|      6|   Lucas|       Taua|            Ceará|     05-09-1984|            1|   39|
|     04|Humberto|Pato Branco|Rio Grande do Sul|     13-11-1964|            2|   59|
|    005|  Isaias|Pato Branco|Rio Grande do Sul|     07-07-2002|            2|   21|
|    001|    José|   Anápolis|        São Paulo|     01-09-1900|            3|  123|
|     02|    Igor|   Anápolis|        São Paulo|     11-09-1977|            3|   46|
|      3|Leonardo|   Anápolis|        São Paulo|     21-12-2000|            3|   23|
+-------+--------+-----------+-----------------+---------------+-------------+-----+



In [0]:
df_cliente = df_cliente.withColumn('cod_cli', lpad(col("cod_cli"), 3, '0'))

In [0]:
df_cliente.show()

+-------+--------+-----------+-----------------+---------------+-------------+-----+
|cod_cli|    Nome|  Município|           Estado|data_nascimento|qtd_municipio|Idade|
+-------+--------+-----------+-----------------+---------------+-------------+-----+
|    006|   Lucas|       Taua|            Ceará|     05-09-1984|            1|   39|
|    004|Humberto|Pato Branco|Rio Grande do Sul|     13-11-1964|            2|   59|
|    005|  Isaias|Pato Branco|Rio Grande do Sul|     07-07-2002|            2|   21|
|    001|    José|   Anápolis|        São Paulo|     01-09-1900|            3|  123|
|    002|    Igor|   Anápolis|        São Paulo|     11-09-1977|            3|   46|
|    003|Leonardo|   Anápolis|        São Paulo|     21-12-2000|            3|   23|
+-------+--------+-----------+-----------------+---------------+-------------+-----+



In [0]:
df_cliente = df_cliente.withColumn('data_atualizacao', current_date())

In [0]:
df_cliente.show()

+-------+--------+-----------+-----------------+---------------+-------------+-----+----------------+
|cod_cli|    Nome|  Município|           Estado|data_nascimento|qtd_municipio|Idade|data_atualizacao|
+-------+--------+-----------+-----------------+---------------+-------------+-----+----------------+
|    006|   Lucas|       Taua|            Ceará|     05-09-1984|            1|   39|      2024-01-26|
|    004|Humberto|Pato Branco|Rio Grande do Sul|     13-11-1964|            2|   59|      2024-01-26|
|    005|  Isaias|Pato Branco|Rio Grande do Sul|     07-07-2002|            2|   21|      2024-01-26|
|    001|    José|   Anápolis|        São Paulo|     01-09-1900|            3|  123|      2024-01-26|
|    002|    Igor|   Anápolis|        São Paulo|     11-09-1977|            3|   46|      2024-01-26|
|    003|Leonardo|   Anápolis|        São Paulo|     21-12-2000|            3|   23|      2024-01-26|
+-------+--------+-----------+-----------------+---------------+-------------+----

In [0]:
def removes (string: str) -> str:
    normalized = unicodedata.normalize('NFD',string)
    return normalized.encode('ascii','ignore').decode('utf8').casefold()

removeudf = udf(lambda z: removes(z))

#ja removendo a coluna
df_cliente = df_cliente.withColumn("Estado",removeudf(col('Estado')))

In [0]:
df_cliente.show()

+-------+--------+-----------+-----------------+---------------+-------------+-----+----------------+
|cod_cli|    Nome|  Município|           Estado|data_nascimento|qtd_municipio|Idade|data_atualizacao|
+-------+--------+-----------+-----------------+---------------+-------------+-----+----------------+
|    006|   Lucas|       Taua|            ceara|     05-09-1984|            1|   39|      2024-01-26|
|    004|Humberto|Pato Branco|rio grande do sul|     13-11-1964|            2|   59|      2024-01-26|
|    005|  Isaias|Pato Branco|rio grande do sul|     07-07-2002|            2|   21|      2024-01-26|
|    001|    José|   Anápolis|        sao paulo|     01-09-1900|            3|  123|      2024-01-26|
|    002|    Igor|   Anápolis|        sao paulo|     11-09-1977|            3|   46|      2024-01-26|
|    003|Leonardo|   Anápolis|        sao paulo|     21-12-2000|            3|   23|      2024-01-26|
+-------+--------+-----------+-----------------+---------------+-------------+----

In [0]:
df_vendas = df_cliente.join(df_produto, df_produto.cod_cli == df_cliente.cod_cli, how='left')

In [0]:
df_vendas.show()

+-------+--------+-----------+-----------------+---------------+-------------+-----+----------------+-------+--------+---------+
|cod_cli|    Nome|  Município|           Estado|data_nascimento|qtd_municipio|Idade|data_atualizacao|cod_cli|cod_prod|     nome|
+-------+--------+-----------+-----------------+---------------+-------------+-----+----------------+-------+--------+---------+
|    001|    José|   Anápolis|        sao paulo|     01-09-1900|            3|  123|      2024-01-26|    001|       1|Televisor|
|    002|    Igor|   Anápolis|        sao paulo|     11-09-1977|            3|   46|      2024-01-26|    002|       1|Televisor|
|    003|Leonardo|   Anápolis|        sao paulo|     21-12-2000|            3|   23|      2024-01-26|    003|       1|Televisor|
|    004|Humberto|Pato Branco|rio grande do sul|     13-11-1964|            2|   59|      2024-01-26|    004|       3|   Violao|
|    005|  Isaias|Pato Branco|rio grande do sul|     07-07-2002|            2|   21|      2024-01

## Fim dos Tratamentos e inicio do Spark SQL

In [0]:
df_cliente.createOrReplaceTempView('tblcliente')

In [0]:
%sql
SELECT * 
FROM   tblcliente cli
WHERE  cli.estado = 'ceara'

cod_cli,Nome,Município,Estado,data_nascimento,qtd_municipio,Idade,data_atualizacao
006,Lucas,Taua,ceara,05-09-1984,1,39,2024-01-26


In [0]:
df_produto.createOrReplaceTempView('tblproduto')

In [0]:
%sql
SELECT * 
FROM   tblproduto

cod_cli,cod_prod,nome
001,1,Televisor
002,1,Televisor
003,1,Televisor
004,3,Violao
005,5,Xbox One
006,10,Geladeira


In [0]:
df_venda_sql = sqlContext.sql(
"""
SELECT * 
FROM   tblcliente cli
LEFT  JOIN tblproduto pro ON pro.cod_cli = cli.cod_cli
WHERE  cli.estado = 'sao paulo'
"""
)

In [0]:
df_venda_sql.show()

+-------+--------+---------+---------+---------------+-------------+-----+----------------+-------+--------+---------+
|cod_cli|    Nome|Município|   Estado|data_nascimento|qtd_municipio|Idade|data_atualizacao|cod_cli|cod_prod|     nome|
+-------+--------+---------+---------+---------------+-------------+-----+----------------+-------+--------+---------+
|    001|    José| Anápolis|sao paulo|     01-09-1900|            3|  123|      2024-01-26|    001|       1|Televisor|
|    002|    Igor| Anápolis|sao paulo|     11-09-1977|            3|   46|      2024-01-26|    002|       1|Televisor|
|    003|Leonardo| Anápolis|sao paulo|     21-12-2000|            3|   23|      2024-01-26|    003|       1|Televisor|
+-------+--------+---------+---------+---------------+-------------+-----+----------------+-------+--------+---------+



## Inicio Python

In [0]:
lst_frutas = ['Abacaxi', 'Banana', 'Limão', 'framboesa', 'Morango', 'Uva', 'Pera']
dic_frutas = {'1': 'Abacaxi', '2': 'Banana', '3': 'Limão', '4': 'framboesa', '5': 'Morango', '6': 'Uva', '7': 'Pera'}
tup_frutas = ('Abacaxi', 'Banana', 'Limão', 'framboesa', 'Morango', 'Uva', 'Pera')

In [0]:
print(lst_frutas)
print(dic_frutas)
print(tup_frutas)

['Abacaxi', 'Banana', 'Limão', 'framboesa', 'Morango', 'Uva', 'Pera']
{'1': 'Abacaxi', '2': 'Banana', '3': 'Limão', '4': 'framboesa', '5': 'Morango', '6': 'Uva', '7': 'Pera'}
('Abacaxi', 'Banana', 'Limão', 'framboesa', 'Morango', 'Uva', 'Pera')


## Iterando sobre lista, dicionário e tuplas

In [0]:
def listar_fruta():
    for i in range(0, len(lst_frutas)):
        print(lst_frutas[i])    

In [0]:
listar_fruta()

Abacaxi
Banana
Limão
framboesa
Morango
Uva
Pera


In [0]:
def listar_frutas_dict(dicionario):
    for chave, valor in dicionario.items():
        print(f'Chave: {chave}, Valor: {valor}')  

In [0]:
listar_frutas_dict(dic_frutas)

Chave: 1, Valor: Abacaxi
Chave: 2, Valor: Banana
Chave: 3, Valor: Limão
Chave: 4, Valor: framboesa
Chave: 5, Valor: Morango
Chave: 6, Valor: Uva
Chave: 7, Valor: Pera


In [0]:
def listar_frutas_tupla(tupla):
    for fruta in tupla:
        print(fruta)

In [0]:
listar_frutas_tupla(tup_frutas)

Abacaxi
Banana
Limão
framboesa
Morango
Uva
Pera


## Manipulando os dados (Lista e Dicionários)

In [0]:
def excluir_item_lista(fruta):
  if fruta in lst_frutas:
    lst_frutas.remove(fruta)
  return lst_frutas

In [0]:
excluir_item_lista('Limão')

Out[169]: ['Abacaxi', 'Banana', 'framboesa', 'Morango', 'Uva', 'Pera']

In [0]:
def excluir_item_dicionario(chave):
  del dic_frutas[chave]
  return dic_frutas

In [0]:
excluir_item_dicionario('3')

Out[171]: {'1': 'Abacaxi',
 '2': 'Banana',
 '4': 'framboesa',
 '5': 'Morango',
 '6': 'Uva',
 '7': 'Pera'}

In [0]:
def adicionar_item_lista(fruta):
  if fruta not in lst_frutas:
    lst_frutas.append(fruta)
  return lst_frutas

In [0]:
adicionar_item_lista('Amora')

Out[173]: ['Abacaxi', 'Banana', 'framboesa', 'Morango', 'Uva', 'Pera', 'Amora']

In [0]:
def adicionar_fruta(dicionario, nova_fruta):
    proximo_indice = str(max(map(int, dicionario.keys()), default=0) + 1)
    if nova_fruta not in dicionario.values():
        dicionario[proximo_indice] = nova_fruta
    return dicionario

In [0]:
adicionar_fruta(dic_frutas, 'Laranja')

Out[178]: {'1': 'Abacaxi',
 '2': 'Banana',
 '4': 'framboesa',
 '5': 'Morango',
 '6': 'Uva',
 '7': 'Pera',
 '8': 'Maçã',
 '9': 'Laranja'}